overlay

In [ ]:
import tkinter as tk
import pyautogui
from PIL import Image, ImageTk

# สร้างหน้าต่างหลัก
root = tk.Tk()
root.attributes("-topmost", True)  # ให้หน้าต่างอยู่ด้านบนสุด
root.overrideredirect(True)  # เอาขอบหน้าต่างออก (Borderless)
root.geometry("1400x100+271+865")  # ขนาดและตำแหน่งหน้าต่าง (กว้าง x สูง + ตำแหน่ง X + ตำแหน่ง Y)
root.configure(bg="black")  # ตั้งค่าสีพื้นหลังโปร่งใส
# root.wm_attributes("-transparentcolor", "black")
# ข้อความที่จะแสดง
text_label = tk.Label(root, text="🌟 แสดงข้อความ Overlay 🌟", font=("tahoma", 24), fg="white", bg="black")
text_label.pack(expand=True)

# ฟังก์ชันให้หน้าต่างสามารถลากได้
def move_window(event):
    root.geometry(f"+{event.x_root}+{event.y_root}")
    print(event.x_root, event.y_root)

# จับ event การลากหน้าต่าง
text_label.bind("<B1-Motion>", move_window)

# เริ่มต้นแสดงหน้าต่าง
root.mainloop()


sc

In [ ]:
import mss
import numpy as np
import cv2

with mss.mss() as sct:
    monitor = {"top": 970, "left": 200, "width": 1400, "height": 180}
    screenshot = sct.grab(monitor)

    # แปลงภาพเป็น OpenCV
    img = np.array(screenshot)
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

    # แสดงภาพที่จับได้
    cv2.imshow("Captured Region", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


translate sub ui

In [3]:
import tkinter as tk
import pytesseract
import mss
import numpy as np
import cv2
import requests
from PIL import Image, ImageTk
import keyboard

# ตั้งค่าตำแหน่งของ Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# ตั้งค่า API สำหรับแปลภาษา
TRANSLATE_API_URL = "http://localhost:8000/translate"  # หรือใช้ Google Translate API

# สร้างหน้าต่างหลัก
root = tk.Tk()
root.attributes("-topmost", True)  # ให้หน้าต่างอยู่ด้านบนสุด
root.overrideredirect(True)  # เอาขอบหน้าต่างออก (Borderless)
root.geometry("1400x100+271+813")  # ขนาดและตำแหน่งหน้าต่าง (กว้าง x สูง + ตำแหน่ง X + ตำแหน่ง Y)
root.configure(bg="black")  # ตั้งค่าสีพื้นหลังโปร่งใส
root.wm_attributes("-transparentcolor", "black")

# ข้อความที่จะแสดง
translated_text = tk.StringVar()

text_label = tk.Label(root, textvariable=translated_text, font=("tahoma", 20), fg="white", bg="black")
text_label.pack(expand=True)

def enhance_image(img):
    """เพิ่มคอนทราสต์และทำให้ภาพคมชัดขึ้น"""
    # ปรับคอนทราสต์
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
    l, a, b = cv2.split(lab)
    l = cv2.equalizeHist(l)  # ปรับ histogram
    lab = cv2.merge([l, a, b])
    img = cv2.cvtColor(lab, cv2.COLOR_Lab2BGR)

    # การทำให้ภาพคมชัด
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])  # Kernel สำหรับ sharpening
    img = cv2.filter2D(img, -1, kernel)

    # ปรับแสง
    img = cv2.convertScaleAbs(img, alpha=1.5, beta=0)
    return img

def preprocess_image(img):
    """ทำการแปลงภาพเพื่อเพิ่มความชัดเจน"""
    # แปลงเป็นขาวดำ
    img = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)  # แปลงเป็น grayscale
    # ใช้ Otsu thresholding เพื่อแปลงภาพเป็น binary
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return img


def capture_screen():
    with mss.mss() as sct:
        # เลือกพื้นที่ของหน้าจอ (แก้ให้ตรงกับตำแหน่งซับเกม)
        monitor = {"top": 900, "left": 200, "width": 1400, "height": 200}
        screenshot = sct.grab(monitor)

        # แปลงภาพเป็น OpenCV
        img = np.array(screenshot)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY)  # แปลงเป็นขาวดำ
        img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]  # เพิ่ม contrast
        
        # ใช้ OCR ดึงข้อความ
        text = pytesseract.image_to_string(img, lang="eng")  # เปลี่ยน "eng" เป็น "tha" ถ้าเป็นภาษาไทย
        return text.strip()

def translate_text(text):
    """ส่งข้อความไปยัง API แปลภาษา"""
    if not text:
        return ""
    data = {"text": text, "to": "th"}  # แปลเป็นภาษาไทย
    response = requests.post(TRANSLATE_API_URL, json=data)
    if response.status_code == 200:
        return response.json().get("translatedText", "แปลไม่ได้")
    return "เกิดข้อผิดพลาด"
    

def update_subtitles():
    """อัพเดตซับไตเติลเกมแบบ Realtime"""
    text = capture_screen()
    if text:     
        translated_text.set(translate_text(text))  # แสดงข้อความที่แปลแล้ว
    root.after(250, update_subtitles)  # อัพเดตทุก 1 วินาที

# ฟังก์ชันปิดโปรแกรม
def close_program(event=None):
    root.destroy()

# ฟังก์ชันให้หน้าต่างสามารถลากได้
def move_window(event):
    root.geometry(f"+{event.x_root}+{event.y_root}")
    print(event.x_root, event.y_root)

# จับ event การลากหน้าต่าง
text_label.bind("<B1-Motion>", move_window)

# จับปุ่ม F1 เพื่อปิดโปรแกรม
keyboard.add_hotkey('F1', close_program)

# เริ่มจับภาพและแปล
update_subtitles()

root.mainloop()


: 

sound read

In [ ]:
import speech_recognition as sr

# เลือกไมค์ (เปลี่ยน index ตามอุปกรณ์ที่ต้องการ)
mic_index = 1  # ลองเปลี่ยนเป็น 1, 2, ... ถ้าไม่ได้ยิน

# เริ่มฟังเสียง
recognizer = sr.Recognizer()
with sr.Microphone(device_index=mic_index) as source:
    recognizer.adjust_for_ambient_noise(source, duration=2)  # ปรับเสียงรบกวน
    print("🎤 กำลังฟัง... พูดอะไรบางอย่าง!")
    while True:
        try:
            audio = recognizer.listen(source, timeout=5, phrase_time_limit=8)
            text = recognizer.recognize_google(audio)
            print(f"📝 ข้อความที่ได้: {text}")
        except sr.UnknownValueError:
            print("❌ ไม่ได้ยินเสียงพูด!")
        except sr.RequestError:
            print("⚠️ มีปัญหาในการเชื่อมต่อกับ Google Speech API")


In [ ]:
import pyaudio

p = pyaudio.PyAudio()
device_count = p.get_device_count()

for i in range(device_count):
    device_info = p.get_device_info_by_index(i)
    print(f"Device {i}: {device_info['name']}")